In [1]:
import astro.config.startup



               Welcome to the Astro-WISE Environment

|01-Feb-19   Planned Groningen database downtime:
|             Tuesdays in 2019: January 8, February 5, March 5, April 2,
                                May 7, June 4, July 2, August 6, September 3,
                                October 8, November 5, December 3
|20-Nov-17   The Astro-WISE documentation has been converted from LaTex to
|            Sphinx. Documentation is now available at doc.astro-wise.org.

Importing Astro-WISE packages. Please wait...

DARMA using PyFITS version 3.4 and NumPy version 1.11.3
Distributed Processing Unit: dpu.hpc.rug.astro-wise.org
[wodan] 2020-05-13T13:24:40 - Warning: astro.main.ESOIDAttributeCalculator is different from stored definition!
Loaded 1 active dpu keys!
Dataserver: ds.astro.rug.astro-wise.org

Current profile: 
- username : awehelmich
- database : db.astro.rug.astro-wise.org
- project  : KIDS
- current privileges : 2 (Project)



In [2]:
from common.database.Context import context
context.set_project('OMEGACAM@VST', 3)

True

In [3]:
from astro.main.SourceList import SourceList
from astro.main.sourcecollection.External import External
#from astro.util.darma.common import new_table
import pyfits
import numpy
import time

In [4]:
hdus = pyfits.open('SA113_binary.fit')

In [5]:
nsources = len(hdus[1].data)
print(nsources)

7989


In [6]:
A = pyfits.Column(name='A', format='F6.4', array=numpy.ones(nsources))
B = pyfits.Column(name='B', format='F6.4', array=numpy.ones(nsources))
POSANG = pyfits.Column(name='POSANG', format='D', array=numpy.zeros(nsources))
FLAG = pyfits.Column(name='FLAG', format='D', array=numpy.zeros(nsources))

In [7]:
columns = hdus[1].data.columns + pyfits.ColDefs([A,B,POSANG,FLAG])

In [8]:
columns.change_name('RA_ICRS', 'RA')
columns.change_name('DE_ICRS', 'DEC')
columns.del_col('Dup')

ColDefs(
    name = 'RA'; format = 'D'; unit = 'deg'; disp = 'F15.11'
    name = 'e_RA_ICRS'; format = 'D'; unit = 'mas'; disp = 'F7.4'
    name = 'DEC'; format = 'D'; unit = 'deg'; disp = 'F15.11'
    name = 'e_DE_ICRS'; format = 'D'; unit = 'mas'; disp = 'F7.4'
    name = 'Source'; format = 'K'; disp = 'I19'
    name = 'Plx'; format = 'D'; unit = 'mas'; disp = 'F10.4'
    name = 'e_Plx'; format = 'E'; unit = 'mas'; disp = 'F7.4'
    name = 'pmRA'; format = 'D'; unit = 'mas/yr'; disp = 'F9.3'
    name = 'e_pmRA'; format = 'E'; unit = 'mas/yr'; disp = 'F6.3'
    name = 'pmDE'; format = 'D'; unit = 'mas/yr'; disp = 'F9.3'
    name = 'e_pmDE'; format = 'E'; unit = 'mas/yr'; disp = 'F6.3'
    name = 'FG'; format = 'E'; unit = 'e-/s'; disp = 'E11.4'
    name = 'e_FG'; format = 'E'; unit = 'e-/s'; disp = 'E11.4'
    name = 'Gmag'; format = 'D'; unit = 'mag'; disp = 'F7.4'
    name = 'e_Gmag'; format = 'D'; unit = 'mag'; disp = 'F6.4'
    name = 'FBP'; format = 'E'; unit = 'e-/s'; disp = 'E1

In [9]:
#columns = pyfits.ColDefs([A,B,POSANG,FLAG, columns['RA'], columns['DEC']])

In [10]:
now = time.time()

In [11]:
# create hdulist
hdulist = pyfits.HDUList()

# create primary hdu
prhdu = pyfits.PrimaryHDU()
prhdu.header.add_history('Created from scratch')
prhdu.header.add_history('Name = %s' % 'scratch')
prhdu.header.add_history('Date = %s' % time.ctime(now))
hdulist.append(prhdu)

# first OBJECTS tablehdu
#tbhdu = new_table(columns)
tbhdu = pyfits.BinTableHDU.from_columns(columns)
tbhdu.name = 'OBJECTS'
hdulist.append(tbhdu)

# fields tablehdu
fields_cols = [pyfits.Column(name='OBJECT_POS', format='1J', array=numpy.array([1])),
               pyfits.Column(name='OBJECT_COUNT', format='1J', array=numpy.array([nsources])),
               pyfits.Column(name='CHANNEL_NAME', format='16A', array=['OMEGACAM']),
               pyfits.Column(name='CHANNEL_NR', format='1J', array=numpy.array([1])),
               pyfits.Column(name='MAPNAXIS', format='1J', array=numpy.array([2])),
               pyfits.Column(name='DATE', format='32A', array=[time.strftime('%FT%H:%M:%S', time.gmtime(now))]),
             pyfits.Column(name='EPOCH', format='1D', array=numpy.array([2000.0]))]
#tbhdu = new_table(columns)
tbhdu = pyfits.BinTableHDU.from_columns(fields_cols)
tbhdu.name = 'FIELDS'
hdulist.append(tbhdu)

In [12]:
filename = 'SA113_SL_READY.fits'

In [13]:
hdulist.writeto(filename, clobber=True)

/home/helmich/Software/miniconda3/envs/astro-env/lib/python3.6/site-packages/pyfits/file.py:372: UserWarning: Overwriting existing file 'SA113_SL_READY.fits'.
  warnings.warn("Overwriting existing file %r." % self.name)


In [14]:
#sourcelist = SourceList()
#sourcelist.sources._set_table_name('SOURCELIST*SOURCES**10')
#sourcelist.catalog = filename
#sourcelist.make_sourcelist_from_catalog()

In [15]:
ext = External()
ext.name = 'SA113_GAIA_patch_Willeke'
ext.make_from_fits(filename)

[wodan] 2020-05-13T13:24:51 - Could not find row_identifier for fits file SA113_SL_READY.fits, faking SeqNr


In [16]:
ext.commit()

In [17]:
ext.store_data()

[wodan] 2020-05-13T13:24:54 - Creating sourcelist_data for External 4473751 .
[wodan] 2020-05-13T13:25:15 - Created SourceList with SLID = 76151051, name = SOURCECOLLECTIONDATA and 7989 sources


In [18]:
dir(ext)

['OBJECT',
 'PROCESS_TIME',
 'SCID',
 'STATUS_COMPARE',
 'STATUS_INSPECT',
 'STATUS_MAKE',
 'STATUS_VERIFY',
 '_IS_CAL',
 '_IS_CONFIG',
 '_IS_RAW',
 '_IS_SCIENCE',
 '_IS_SEQ',
 '_IS_SUPPORT',
 '_SCT',
 '_SOURCELIST_DATA_NAME',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cached_tableconverter_data',
 '_created',
 '_creator',
 '_delete',
 '_delete_post',
 '_desc',
 '_dirty',
 '_exist',
 '_get_onthefly',
 '_get_universe_sourcelist',
 '_inverses',
 '_modified',
 '_origin',
 '_privileges',
 '_project',
 '_publish',
 '_select_generic',
 '_tried_load_kids',
 '_updated',
 '_uptodate',
 '_validate_flags',
 '_version',
 'action',
 'add_sourcelist_data_to_relati

In [ ]:
#sl = (SourceList.SLID == 76151031)[0]

In [ ]:
#sl.sources.get_data()